# Cellpose Batch Processing for TIFF Files
This notebook processes all TIFF files in a specified folder using Cellpose.

In [ ]:
# Import required libraries
import os
import numpy as np
from pathlib import Path
from cellpose import models, io
from skimage import io as skio
import matplotlib.pyplot as plt

os.environ['OMP_NUM_THREADS'] = '1'

In [ ]:
# Configuration
input_folder = "./data/images"  # Change this to your folder path
output_folder = "./data/instance_masks"    # Where to save results
model_type = "cyto3"  # Options: 'cyto', 'cyto2', 'cyto3', 'nuclei'
channels = [0, 0]  # [cytoplasm_channel, nucleus_channel], use [0,0] for grayscale
diameter = None  # Approximate cell diameter in pixels (None for auto-detection)
flow_threshold = 0.4  # Default: 0.4
cellprob_threshold = 0.0  # Default: 0.0

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# Get all TIFF files from folder
tiff_extensions = ['*.tiff', '*.tif', '*.TIF', '*.TIFF']
image_files = []

for ext in tiff_extensions:
    image_files.extend(Path(input_folder).glob(ext))

image_files = sorted([str(f) for f in image_files])
print(f"Found {len(image_files)} TIFF files")
for f in image_files[:5]:  # Show first 5 files
    print(f"  - {os.path.basename(f)}")

In [ ]:
# Initialize Cellpose model
# Use GPU if available (gpu=True), otherwise CPU (gpu=False)
model = models.CellposeModel(gpu=False, model_type=model_type)
print(f"Loaded {model_type} model")
print(f"Using GPU: {model.gpu}")

In [ ]:
# Process all images
results = []

for i, img_path in enumerate(image_files):
    print(f"\nProcessing {i+1}/{len(image_files)}: {os.path.basename(img_path)}")
    
    try:
        # Read image
        img = skio.imread(img_path)
        print(f"  Image shape: {img.shape}")
        
        # Run Cellpose
        masks, flows, styles, diams = model.eval(
            img, 
            diameter=diameter,
            channels=channels,
            flow_threshold=flow_threshold,
            cellprob_threshold=cellprob_threshold
        )
        
        # Count cells
        n_cells = len(np.unique(masks)) - 1  # Subtract background
        print(f"  Detected {n_cells} cells")
        
        # Save results
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        
        # Save masks as TIFF only
        skio.imsave(os.path.join(output_folder, f"{base_name}_masks.tif"), masks.astype(np.uint16))
        
        results.append({
            'filename': os.path.basename(img_path),
            'n_cells': n_cells,
            'diameter_used': diams
        })
        
    except Exception as e:
        print(f"  ERROR: {str(e)}")
        results.append({
            'filename': os.path.basename(img_path),
            'n_cells': 'ERROR',
            'diameter_used': 'ERROR'
        })

print("\n=== Processing Complete ===")